In [1]:
KotlinVersion.CURRENT             // 코틀린 현재 버전 

1.8.0

In [1]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.6.4")

In [2]:
:classpath

Current classpath (16 paths):
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/lib-0.11.0-170.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/api-0.11.0-170.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlin-script-runtime-1.8.0-dev-3517.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlin-reflect-1.7.10.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlin-stdlib-1.7.10.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlin-stdlib-common-1.7.10.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/annotations-13.0.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlinx-serialization-json-jvm-1.3.3.jar
/Users/a06

In [7]:
import kotlinx.coroutines.*
import kotlinx.coroutines.channels.*
import kotlinx.coroutines.flow.*

# 3. 코루틴 채널과 플로우 

## 3- 1 채널

In [6]:
fun main() {
    val channel = Channel<Int>() 
    println(channel.javaClass)

    CoroutineScope(Dispatchers.Default).launch(Dispatchers.Default) { 
        channel.send(100)
    }
    
    runBlocking {

        delay(1000)
        println(channel.receive())
        channel.close()
    }
}

main()

class kotlinx.coroutines.channels.RendezvousChannel
100


In [17]:
fun main() {

    val channel = Channel<Int>() 
    CoroutineScope(Dispatchers.Default).launch { 
        repeat(10) { channel.send(it) } 
    } 

    CoroutineScope(Dispatchers.Default).launch {
        val num = channel.receive()                  // 1 
        println("한개 처리 : $num ")
        var count =0 
        for(x in channel) {                         // 2 
            println("for 순환 : $x ") 
            count++
            if (count > 4) delay(1000)
        } 
    }
    
    CoroutineScope(Dispatchers.Default).launch {
  
        channel.consumeEach {                       // 3 
            println("consume 순환 : $it") } 
    }

}

main()

한개 처리 : 0 
consume 순환 : 1
consume 순환 : 2
consume 순환 : 4
consume 순환 : 5
for 순환 : 3 
consume 순환 : 6
for 순환 : 7 
for 순환 : 9 
consume 순환 : 8


In [18]:
// interface Channel<E> : SendChannel<E> , ReceiveChannel<E>  // 채널은 송신과 수신 채널을 묶는 개념 
// fun <E> Channel(                                           // 채널함수 
//    capacity: Int = RENDEZVOUS,                             // 버퍼 용량 
//    onBufferOverflow: BufferOverflow = BufferOverflow.SUSPEND, 
//    onUndeliveredElement: (E) -> Unit? = null
// ): Channel<E>

fun main() = runBlocking {
        val chan = Channel<Int>()
        launch {
            for (x in 1..5) chan.send(x *x)                   // 값을 보냄
        }
        launch {
            repeat(3) { 
                delay(10)
                println("코루틴1 : " + chan.receive())          // 값을 받음 
            }                                                  // 
        }
        launch {
            repeat(2) { 
                delay(20)
                println("코루틴2 : " +chan.receive())           // 값을 받음 
            }                                                 // 
        }
        delay(200)
        chan.close()                                      // 더 보낼 값이 없음, close
        println("Done!")
}
main()

코루틴1 : 1
코루틴2 : 4
코루틴1 : 9
코루틴1 : 16
코루틴2 : 25
Done!


In [19]:
fun Rendezvous_Channel() {
    runBlocking {
        val basket = Channel<String>()
        launch {                                              // 코루틴에서 순환문으로 전송
            val fruits = listOf("사과", "오렌지")
            for (fruit in fruits) {
                println("송신 1 코루틴 : $fruit")
                 delay(100)
                basket.send(fruit)
            }
            for (fruit in fruits) {                            // 코루틴에서 순환문으로 전송
                println("송신 2 코루틴 : $fruit")
                delay(100)
                basket.send(fruit)
            } 
            basket.close()                                      // 채널 닫음 
        }
        launch(Dispatchers.IO) {                                // 4개의 코뤁틴을 만듬 
            repeat(4) {                                         // 각각 출력 
                delay(100)
                println(Thread.currentThread().name)
                println("수신 코루틴 : ${basket.receive()}")
            }
        }
    }
}

Rendezvous_Channel()

송신 1 코루틴 : 사과
DefaultDispatcher-worker-3
송신 1 코루틴 : 오렌지
수신 코루틴 : 사과
DefaultDispatcher-worker-3
수신 코루틴 : 오렌지
송신 2 코루틴 : 사과
DefaultDispatcher-worker-3
수신 코루틴 : 사과
송신 2 코루틴 : 오렌지
DefaultDispatcher-worker-3
수신 코루틴 : 오렌지


In [20]:
fun buffer_Channel() {  
    runBlocking {
        val basket = Channel<String>(2)                      // 채널 
        val fruits = listOf("바나나", "딸기", "앵두")
        launch {                                             // 송신 고루틴 1
            for (fruit in fruits) {
                println(" 송신 1: $fruit")
                basket.send(fruit)
            }
        }
        launch {                                             // 송신 코루틴 2
            for (fruit in fruits) {
                println(" 송신 2: $fruit")
                basket.send(fruit)
            }
        }
        launch {                                             // 수신 코루틴 
            repeat(6) {
                delay(100)
                println(" 수신 : ${basket.receive()}")
            }
        }  
        delay(3000)
        basket.close()                                        // 채널 닫기 
    }
}
buffer_Channel()

 송신 1: 바나나
 송신 1: 딸기
 송신 1: 앵두
 송신 2: 바나나
 수신 : 바나나
 수신 : 딸기
 송신 2: 딸기
 수신 : 앵두
 송신 2: 앵두
 수신 : 바나나
 수신 : 딸기
 수신 : 앵두


In [21]:
// interface ProducerScope<in E>                          // 생성자스코프 
// : CoroutineScope, SendChannel<E>                       // 코루틴스코프와 센드채널 상속

// fun <E> CoroutineScope.produce(                        // 생산자 빌더 
//    context: CoroutineContext = EmptyCoroutineContext,  // 컨텍스트  
//    capacity: Int = 0,                                  // 용량
//    block: suspend ProducerScope<E>.() -> Unit          // 생산자 스코프의 람다표현식 
// ): ReceiveChannel<E>                                   // 수신 채널 

fun CoroutineScope.produceSquares() = produce {      // 코루틴스코프 내의 확장함수 생성 
    for (x in 1..5) {
        send(x * x)                                  // 값을 하나씩 produce
    }
}

fun main() = runBlocking {
    val squares = produceSquares()                   // 생산자 함수 실행 
    println(squares.javaClass)                       // 수신자 반환
    squares.consumeEach { println(it) }              // 값을 하나씩 consume
    println("Done!")
}

main()

class kotlinx.coroutines.channels.ProducerCoroutine
1
4
9
16
25
Done!


## 3-2 플로우

In [8]:
// interface Flow<out T>

// fun <T> flow(                                     // 플로우 함수 
// block: suspend FlowCollector<T>.() -> Unit)       // 일시중단람다표현식 
// : Flow<T>                                         // 반환형은 플로우 

In [11]:
fun fooList(): List<Int> = listOf(1, 2, 3)             // 리스트 생성 함수 작성 
fun main1() {
    fooList().forEach { value -> 
                        println("리스트 $value ") }      // 리스트를 처리 
}
main1()

리스트 1 
리스트 2 
리스트 3 


In [12]:
fun fooFlow(): Flow<Int> = flow {                      // 일반함수에  flow builder
    for (i in 1..3) {                                  // 순환 
        delay(100)                                     // 일시중단함수 
        emit(i)                                        // 다음 값 내보내기
    }
}
fun main() = runBlocking<Unit> {                       // 런블러킹 빌더 처리 
                                                       // 동시에 코루틴을 사용하여 메인 스레드가 차단되는지 확인
    launch {                                           // 코루틴 처리 
        for (k in 1..3) {
            println("블럭여부 확인 $k")
            delay(100)                                 // 지연함수 
        }
    }
                                                       // flow collect
    fooFlow().collect { value ->                       // 플로우 종단함수 
                        println("플로우: $value ") } 
    println("리스트로 변환 : " + fooFlow().toList())      // 리스트로 변환처리
}
main()

블럭여부 확인 1
플로우: 1 
블럭여부 확인 2
플로우: 2 
블럭여부 확인 3
플로우: 3 
리스트로 변환 : [1, 2, 3]


In [13]:
fun foo(): Flow<Int> = flow {                         // 플로우 빌더는 일시중단함수로 정의하지 않음 
    println("플로우 시작")                               // 
    for (i in 1..3) {
        delay(100)
        emit(i)                                       // 데이터 송신
    }
}

fun main() = runBlocking<Unit> {
    println("플로우 스코프 만듬")
    val flow = foo()                                   // 플로우 함수 실행
    println(flow.javaClass)                            // 플로우 객체로 변환

    flow.collect { value -> println(value) }           // 플로우  처리 
    println("집합 변환 : " + flow.toSet())               // 집합으로 변환 
    
    launch {
        println("코루틴 내부에서 호출")
        flow.collect { value -> println(value) }       // 플로우 종단함수 
    }
}

main()

플로우 스코프 만듬
class kotlinx.coroutines.flow.SafeFlow
플로우 시작
1
2
3
플로우 시작
집합 변환 : [1, 2, 3]
코루틴 내부에서 호출
플로우 시작
1
2
3


In [14]:
fun convert(request: Int): String {
    return "일반함수변환 $request"                       // 플로우 값 매핑
}

suspend fun convertFlow(request: Int): String {
    //delay(1000)                                     // 1초 대기
    return "일시중단함수변환 $request"                    // 플로우 값 매핑 
}

fun main() = runBlocking<Unit> {
    val foo = (1..3).asFlow()                         // 범위를 플로우로 변환 
                       .flowOn(Dispatchers.IO)        // 플로우의 컨텍스트 변환
    foo.map { it -> convertFlow(it) }                 // 플로우에도 map 으로 변환 처리 가능 
        .collect { it -> println(it) }                // 플로우 종단함수 
        
    foo.map { it -> convert(it) }                     // 플로우에도 map 으로 변환 처리 가능 
        .collect { it -> println(it) }                // 플로우 종단함수 

}

main()

일시중단함수변환 1
일시중단함수변환 2
일시중단함수변환 3
일반함수변환 1
일반함수변환 2
일반함수변환 3


In [15]:
fun convert(request: Int): String {                   // 일반함수 정의 
    return "일반함수변환 $request"                       // 플로우 값 매핑
}

suspend fun convertFlow(request: Int): String {       // 일시중단함수 정의 
    delay(100)                                        // 1초 대기
    return "일시중단함수변환 $request"                     // 플로우 값 매핑 
}


fun main() = runBlocking<Unit> {
    val foo = (1..3).asFlow()                          // 배열 -> Flow 변환
    
    foo.transform { it ->                              // 변환 메소드로 여러 개를 변환 
        emit("문자열로 변환 $it")                         // emit으로 flow 처리
        emit(convert(it))                              // emit으로 flow 처리
        emit(convertFlow(it))                          // emit으로 flow 처리
    }.collect { it -> println(it) }                    // 플로우 종단함수 
}

main()

문자열로 변환 1
일반함수변환 1
일시중단함수변환 1
문자열로 변환 2
일반함수변환 2
일시중단함수변환 2
문자열로 변환 3
일반함수변환 3
일시중단함수변환 3


In [16]:
fun main() { 
    
    runBlocking<Unit> {                                  // 런블러킹 스코프

        val job = launch {                               // 코루틴 빌더 
            delay(200)
            (1..5).asFlow().collect { println("$it") }   // 플로우 처리 
        } 
        
        delay(300)
        println(" 플로우 취소하기 ")                         //  플로우 중단
        job.cancel()
    }
}

main()

1
2
3
4
5
 플로우 취소하기 
